<div>
<left>
    <img align="left" src="https://upload.wikimedia.org/wikipedia/commons/0/07/Logo_INGV_dal_2018.png" width="100" width="100"/>
</left>
</div>    


<div>
<right>
    <img align="right" src="./Logo_MACMAP.png" width="100" width="100"/>
</right>
</div>

<h1><center>QUERYING and RETRIVING XBT DATA FROM ERDDAP</center></h1>

*Authors*: Claudia Fratianni (claudia.fratianni@ingv.it), Paolo Frizzera (paolo.frizzera@ingv.it), Simona Simoncelli (simona.simoncelli@ingv.it)
***
# Description
The objective of this notebook is to allow querying and retrieving __Reprocessed XBT dataset (V2) in the Ligurian and Thyrrhenian Seas (1999-2019)__ by using __Erddapy__ package, that takes advantage of ERDDAP’s RESTful web services and creates the ERDDAP URL for any request, like searching for datasets, acquiring metadata, downloading the data, etc. 



***
 
This notebook has been developed within the framework of the Multidisciplinary Analysis of Climate change indicators in the Mediterranean And Polar regions (__MACMAP__) project funded by Istituto Nazionale di Geofisica e Vulcanologia (Environment Department).

# QUICK introduction 

__Erddapy__ package can be installed with  <span style='color: #e74c3c; font-family: monospace; background-color: #F7F2F4;font-size:12pt;'>pip</span>

pip install erddapy

# QUERYING ERDDAP servers from PYTHON

In [1]:
import urllib.request
import json
import pandas as pd
import requests
import numpy as np
import xarray as xr

from netCDF4 import Dataset
from erddapy import ERDDAP

#### 1. Initiate the ERDDAP URL constructor for a server ( erddapy server object)

In [2]:
e=ERDDAP(
  server="http://oceano.bo.ingv.it/erddap",
  protocol="tabledap"
)

#### 2. Populate the object with a dataset id and its constraints. 

In [3]:
dataset_id = "REP_XBT_1999_2019_v2"
e.dataset_id = dataset_id
e.constraints = {"time<=": "1999-09-21"} #! This is only an example you can add more constraints 

#### 3. Generate the download url, choosing the fileType which meets your needs and is easiest to use

In [4]:
format="mat"
du=e.get_download_url(response=format)
print(du)

#### 4. Download interested dataset

In [5]:
#response=requests.get(url)
#with open("REP_XBT_1999_2019_v2."+format, "wb") as fout:
#    fout.write(response.content)

#### 5. Download data into Python

It is also possible to load the query response directly into Python, both using to_pandas and to_xarray erddapy methods in order to retrive data and associated metadata. 

In [6]:
dp = e.to_pandas()

In [7]:
# print the dataframe to check what data is in there specifically.
dp.head()

,CALIB (degree_C),depth (m),DEPTH_COR (meters),DEPTH_COR_FLAGS_QC,DEPTH_COR_INT (meters),DEPTH_COR_INT_SEADATANET_QC,DEPTH_FLAGS_QC,DEPTH_INT (meters),DEPTH_INT_SEADATANET_QC,DEPTH_TEST_QC,...,TEMPET01_COR_INT_SEADATANET_QC,TEMPET01_FLAGS_QC,TEMPET01_INT (degree_C),TEMPET01_INT_SEADATANET_QC,TEMPET01_TEST_QC,time (UTC),TIME_SEADATANET_QC,cruise_id,profile_id,url_metadata
0,NaN,0.00,0.254125,1.0,1.0,1.0,1,1.0,1.0,1.0,...,1.0,2,23.826,1.0,49,1999-09-20T19:54:55Z,1,MFSPP_990920,MFSPP_990920_001,http://erddap-dev.bo.ingv.it/erddap/tabledap/T...
1,NaN,0.67,0.931556,1.0,2.0,1.0,1,2.0,1.0,1.0,...,1.0,2,23.850,1.0,49,1999-09-20T19:54:55Z,1,MFSPP_990920,MFSPP_990920_001,http://erddap-dev.bo.ingv.it/erddap/tabledap/T...
2,NaN,1.34,1.619221,1.0,3.0,1.0,1,3.0,1.0,1.0,...,1.0,1,23.848,1.0,49,1999-09-20T19:54:55Z,1,MFSPP_990920,MFSPP_990920_001,http://erddap-dev.bo.ingv.it/erddap/tabledap/T...
3,NaN,2.00,2.306852,1.0,4.0,1.0,1,4.0,1.0,1.0,...,1.0,1,23.869,1.0,49,1999-09-20T19:54:55Z,1,MFSPP_990920,MFSPP_990920_001,http://erddap-dev.bo.ingv.it/erddap/tabledap/T...
4,NaN,2.68,2.994453,1.0,5.0,1.0,1,5.0,1.0,1.0,...,1.0,1,23.853,1.0,49,1999-09-20T19:54:55Z,1,MFSPP_990920,MFSPP_990920_001,http://erddap-dev.bo.ingv.it/erddap/tabledap/T...


In [8]:
dr = e.to_xarray()

#### 6. Retrieve single profile with corresponding detailed metadata information

Details metadata of each profile can be retrieve throught __profile_id__ and __url_metadata__ variables. 

In [9]:
profile=np.unique((dp['profile_id'].values))
print(profile)

url=np.unique((dp['url_metadata'].values))

['MFSPP_990920_001' 'MFSPP_990920_002' 'MFSPP_990920_003'
 'MFSPP_990920_004' 'MFSPP_990920_005' 'MFSPP_990920_006'
 'MFSPP_990920_007']


In [10]:
for p in profile:
 ddp=(dp[dp['profile_id']==p]['depth (m)'].values)
 diffs=np.diff(ddp)<0
 indexes = np.where(diffs)[0]+1

# MERGE DATA AND VARIABLE METADATA
 for pp in range(len(list(dp[dp['profile_id']==p]))):
  varname_orig=(list((dp))[pp].split()[0])
  varname=list(dp)[pp]
  if (varname_orig) in dr.data_vars:
    prof_size=(np.size(dp[dp['profile_id']==p][varname]))
    var_val=dp[dp['profile_id']==p][varname].values
    den=(len(indexes)+1)
    num1=(np.size(ddp))
    nn1=(int(num1/den))
    if (varname_orig) in ["TEMPET01_TEST_QC","DEPTH_TEST_QC"]:
     var_res=(var_val.reshape(den,nn1))
     ds=xr.DataArray(var_res,dims=['TST_T','MAXZ'],coords={'TST_T': np.arange(0,den),'MAXZ': np.arange(0,nn1)},name=varname_orig,attrs=dr.data_vars[varname_orig].attrs)
     ds.to_netcdf(p+'.nc','r+')
     ds.close()
    else:
     var_res=(var_val[0:nn1])
     ds=xr.DataArray(var_res,dims=['MAXZ'],coords={'MAXZ': np.arange(0,nn1)},name=varname_orig,attrs=dr.data_vars[varname_orig].attrs)
     ds.to_netcdf(p+'.nc','r+')
     ds.close()
  else:
    if varname_orig in "longitude" or varname_orig in "latitude" or varname_orig in "time":
     var_val=dp[dp['profile_id']==p][varname].values[0]
     ds=xr.DataArray(var_val,dims=['INSTANCE'],coords={'INSTANCE': np.arange(0,1)},name=varname_orig, attrs=dr[varname_orig].attrs)
     ds.to_netcdf(p+'.nc','r+')
     ds.close()
        
 index=[idx for idx, s in enumerate(url) if p in s][0]
 new_url=url[index].replace('htmlTable','json')
 response=urllib.request.urlopen(new_url)
 string=response.read()
 json_obj = json.loads(string)
 element=(json_obj['table'].get('columnNames'))
 element_values=(json_obj['table'].get('rows')[0])
 infor=pd.DataFrame({'Elements':element,'Values':element_values})
 ncid=Dataset(p+'.nc','r+')
 for gl in range(len(element)):
  gl_at='ncid.'+element[gl]+' ="'+str(element_values[gl])+'"'
  exec(gl_at)
 ncid.close()
 print('#####  METADATA ASSOCIATED TO PROFILE '+p)

#####  METADATA ASSOCIATED TO PROFILE MFSPP_990920_001
#####  METADATA ASSOCIATED TO PROFILE MFSPP_990920_002
#####  METADATA ASSOCIATED TO PROFILE MFSPP_990920_003
#####  METADATA ASSOCIATED TO PROFILE MFSPP_990920_004
#####  METADATA ASSOCIATED TO PROFILE MFSPP_990920_005
#####  METADATA ASSOCIATED TO PROFILE MFSPP_990920_006
#####  METADATA ASSOCIATED TO PROFILE MFSPP_990920_007
